In [0]:
import string
import re
import gc
import numpy as np
import pandas as pd
import random
from sklearn import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from scipy.sparse import hstack, vstack, csr_matrix
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb
import lightgbm as lgb
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk import punkt
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
import pandas_profiling
import lightgbm as lgb

In [0]:
nltk.download('stopwords')
nltk.download('wordnet')
pd.options.display.max_columns = None
stop_words = stopwords.words('english')
punctuation = string.punctuation
stop_words = list(set(stopwords.words('english')))
id_column = "id"
missing_token = " UNK "
np.random.seed(2018)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
!pip install kaggle


In [0]:
from google.colab import files


In [0]:
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anupamshandilya92","key":"5a4efdbf2d37f0db0b5e6fc5d6e3965d"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [0]:
!pip install --upgrade --force-reinstall --no-deps kaggle



     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=c90f59e3bcfe1c31c3f5d5e1494e71748c68984b717799cf516b7e709c712648
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


In [0]:
files.upload()

Saving donorschoose-application-screening.zip to donorschoose-application-screening.zip


In [0]:
!ls

In [0]:
!unzip donorschoose-application-screening.zip

In [0]:
!unzip train.zip
!unzip test.zip
!unzip resources.zip

Archive:  train.zip
  inflating: train.csv               
Archive:  test.zip
  inflating: test.csv                
Archive:  resources.zip
  inflating: resources.csv           


In [0]:
!ls

donorschoose-application-screening.zip	sample_data	       train.csv
kaggle.json				sample_submission.zip  train.zip
resources.csv				test.csv
resources.zip				test.zip


In [0]:
train = pd.read_csv("/content/train.csv", parse_dates=["project_submitted_datetime"])
test = pd.read_csv("/content/test.csv", parse_dates=["project_submitted_datetime"])
resources = pd.read_csv("/content/resources.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182080 entries, 0 to 182079
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   id                                            182080 non-null  object        
 1   teacher_id                                    182080 non-null  object        
 2   teacher_prefix                                182076 non-null  object        
 3   school_state                                  182080 non-null  object        
 4   project_submitted_datetime                    182080 non-null  datetime64[ns]
 5   project_grade_category                        182080 non-null  object        
 6   project_subject_categories                    182080 non-null  object        
 7   project_subject_subcategories                 182080 non-null  object        
 8   project_title                                 182080 n

In [0]:
resources.head()

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95


In [0]:
resources['total_price'] = resources['quantity']*resources['price']

agg_rc = resources.groupby('id')\
                    .agg({'description':'count', 'quantity':'sum', 'price':'sum', 'total_price':'sum'})\
                    .rename(columns={'description':'items'})

for func in ['min', 'max', 'mean']:
    agg_rc_temp = resources.groupby('id')\
                    .agg({'quantity':func, 'price':func, 'total_price':func})\
                    .rename(columns={'quantity':func+'_quantity',
                                     'price':func+'_price',
                                     'total_price':func+'_total_price'})\
                    .fillna(0)
    agg_rc = agg_rc.join(agg_rc_temp)

agg_rc = agg_rc.join(resources.groupby('id')\
                     .agg({'description':lambda x:' '\
                           .join(x.values.astype(str))})\
                     .rename(columns={'description':'resource_description'}))

agg_rc.head()

,items,quantity,price,total_price,min_quantity,min_price,min_total_price,max_quantity,max_price,max_total_price,mean_quantity,mean_price,mean_total_price,resource_description
id,,,,,,,,,,,,,,
p000001,4,7,459.56,833.63,1,23.99,47.98,2,261.08,522.16,1.750000,114.890000,208.407500,"Cap Barbell 300 Pound Olympic Set, Grey Cap Ba..."
p000002,14,21,515.89,630.28,1,8.46,9.95,4,134.90,134.90,1.500000,36.849286,45.020000,10 Sony Headphones (BUY 9 GET 1 FREE) Belkin 6...
p000003,4,4,298.97,298.97,1,39.99,39.99,1,169.00,169.00,1.000000,74.742500,74.742500,EE820X - Phonemic Awareness Instant Learning C...
p000004,95,98,1113.69,1126.22,1,1.60,3.20,2,401.54,401.54,1.031579,11.723053,11.854947,A Bad Case of the Giggles Poems That Will Make...
p000005,4,8,485.99,702.31,1,54.08,54.08,3,323.75,323.75,2.000000,121.497500,175.577500,"Fitbit Zip Wireless Activity Tracker, Lime Fit..."


In [0]:
#Merging train and test
train['Ind'] = 'Train'
test['project_is_approved'] = 0
test['Ind'] = 'Test'
df = pd.concat([train, test], axis=0)
df = df.join(agg_rc, on='id')
del(train,test)

In [0]:
df.head()


,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,Ind,items,quantity,price,total_price,min_quantity,min_price,min_total_price,max_quantity,max_price,max_total_price,mean_quantity,mean_price,mean_total_price,resource_description
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1,Train,2,6,299.98,899.94,3,149.99,449.97,3,149.99,449.97,3.0,149.990,449.970,Apple - iPod nano� 16GB MP3 Player (8th Genera...
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0,Train,1,20,20.00,400.00,20,20.00,400.00,20,20.00,400.00,20.0,20.000,400.000,Reebok Girls' Fashion Dance Graphic T-Shirt - ...
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1,Train,1,1,469.99,469.99,1,469.99,469.99,1,469.99,469.99,1.0,469.990,469.990,3doodler Start Full Edu Bundle
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0,Train,5,5,684.47,684.47,1,18.95,18.95,1,354.99,354.99,1.0,136.894,136.894,BALL PG 4'' POLY SET OF 6 COLORS BALL PLAYGROU...
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1,Train,1,2,355.50,711.00,2,355.50,711.00,2,355.50,711.00,2.0,355.500,711.000,Crown Berkey Water Filter With 2 Black and 2 P...


In [0]:
df["Year"] = df["project_submitted_datetime"].dt.year
df["Month"] = df["project_submitted_datetime"].dt.month
df['Weekday'] = df['project_submitted_datetime'].dt.weekday
df["Month_Day"] = df['project_submitted_datetime'].dt.day
df["Weekend_Flag"]=np.where((df['Weekday']==5) |(df['Weekday']==6),1,0)

In [0]:

#Setting up a dummy
testDf = df.copy()
testDf.loc[testDf['Ind']=='train', 'Ind'] =0 
testDf.loc[testDf['Ind']=='test', 'Ind'] =1
testDf.loc[testDf['Ind']==1, 'project_is_approved'] = 0

testDf['approvedValue'] = testDf['total_price'] * testDf['project_is_approved']
num =  (testDf.groupby(['teacher_id', 'project_submitted_datetime'])['approvedValue'].sum().groupby(level = [0]).shift(1).cumsum())
numdf = num.reset_index()
#numdf[0].replace(np.nan, 0, inplace=True)
numdf['approvedValue'] = numdf.groupby(['teacher_id'])['approvedValue'].shift(1).replace(np.nan, 0)

#Join it with main table
df = df.merge(numdf, on=['teacher_id', 'project_submitted_datetime'])

In [0]:
df['four_essays'] = 1
df.loc[pd.isnull(df.project_essay_3), 'four_essays'] = 0

In [0]:
def merge_essays_one(x):
    essay_one = ''
    if (x.project_essay_3 is None or not isinstance(x.project_essay_3,str)) and (x.project_essay_4 is None or not isinstance(x.project_essay_4,str)):
        essay_one  = essay_one+x.project_essay_1
    else:
        essay_one  = essay_one+str(x.project_essay_1)+' '+str(x.project_essay_2)
    return essay_one

df['project_essay_1'] = df.apply(merge_essays_one,axis=1)

In [0]:
def merge_essays_two(x):
    essay_two = ''
    if (x.project_essay_3 is None or not isinstance(x.project_essay_3,str)) and (x.project_essay_4 is None or not isinstance(x.project_essay_4,str)):
        essay_two  = essay_two+x.project_essay_2
    else:
        essay_two  = essay_two+str(x.project_essay_3)+' '+str(x.project_essay_4)
    return essay_two

df['project_essay_2'] = df.apply(merge_essays_two,axis=1)

In [0]:
df.fillna(value = {'project_essay_3': " ", "project_essay_4": " "}, inplace=True)
df['text'] = df.apply(lambda row: ' '.join([
    str(row['project_essay_1']),
    str(row['project_essay_2']),
    str(row['project_essay_3']),
    str(row['project_essay_4']),
    str(row['project_resource_summary']),
    str(row['project_title']),
    str(row['resource_description'])]), axis=1)

punctuation = string.punctuation

# extract features from text
df['char_count'] = df['text'].apply(len)
df['word_count'] = df['text'].apply(lambda x: len(x.split()))
df['word_density'] = df['char_count'] / (df['word_count']+1)
df['punctuation_count'] = df['text'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation)))
df['title_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
df['upper_case_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
df['stopword_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.lower() in stop_words]))

Sentiment


In [0]:

df['title_len'] = df['project_title'].apply(lambda x: len(x.split()))
df['e1_len'] = df['project_essay_1'].apply(lambda x: len(x.split()))
df['e2_len'] = df['project_essay_2'].apply(lambda x: len(x.split()))
df['ressum_len'] = df['project_resource_summary'].apply(lambda x: len(x.split()))

In [0]:

#Project Title - Length, Sentiment
#df['title_len'] = df['project_title'].apply(len)
df['title_polarity'] = df['project_title'].apply(lambda x: TextBlob(x).polarity)
df['title_subjectivity'] = df['project_title'].apply(lambda x: TextBlob(x).subjectivity)

In [0]:
#Resource - Length, Sentiment
df['ressum_polarity'] = df['project_resource_summary'].apply(lambda x: TextBlob(x).polarity)
df['ressum_subjectivity'] = df['project_resource_summary'].apply(lambda x: TextBlob(x).subjectivity)

In [0]:
#Project Essay - Length, Sentiment
df['e1_polarity'] = df['project_essay_1'].apply(lambda x: TextBlob(x).polarity)
df['e1_subjectivity'] = df['project_essay_1'].apply(lambda x: TextBlob(x).subjectivity)
df['e2_polarity'] = df['project_essay_2'].apply(lambda x: TextBlob(x).polarity)
df['e2_subjectivity'] = df['project_essay_2'].apply(lambda x: TextBlob(x).subjectivity)

In [24]:
%%time
!pip install autocorrect
from nltk.stem.wordnet import WordNetLemmatizer
from autocorrect import spell  # as spell checker and corrector
L = WordNetLemmatizer()
p = PorterStemmer()
def wordPreProcess(sentence):
    return ' '.join([p.stem(x.lower()) for x in re.split('\W', sentence) if len(x) >= 1])

def getTextFeatures(T, Col, max_features=10000, ngrams=(1,2), verbose=True):
    if verbose:
        print('processing: ', Col)
    vectorizer = CountVectorizer(stop_words=None,
                                 preprocessor=wordPreProcess,
                                 max_features=max_features,
                                 binary=True,
                                 ngram_range=ngrams)
    X = vectorizer.fit_transform(T[Col])
    return X, vectorizer.get_feature_names()

n_es1, n_es2, n_prs, n_rd, n_pt = 3000, 8000, 2000, 3000, 1000
X_es1, feat_es1 = getTextFeatures(df, 'project_essay_1', max_features=n_es1)
X_es2, feat_es2 = getTextFeatures(df, 'project_essay_2', max_features=n_es2)
X_prs, feat_prs = getTextFeatures(df, 'project_resource_summary', max_features=n_prs)
X_rd, feat_rd = getTextFeatures(df, 'resource_description', max_features=n_rd, ngrams=(1,3))
X_pt, feat_pt = getTextFeatures(df, 'project_title', max_features=n_pt)

X_txt = hstack((X_es1, X_es2, X_prs, X_rd, X_pt))
del X_es1, X_es2, X_prs, X_rd, X_pt

     |████████████████████████████████| 1.8MB 2.8MB/s 
  Created wheel for autocorrect: filename=autocorrect-1.1.0-cp36-none-any.whl size=1810772 sha256=e952773427ad78b00df2f1d25f48b2588cf7cb64ea9b0901ad8d587e66999e99
  Stored in directory: /root/.cache/pip/wheels/78/7f/b1/527522820ae623df6a2dbe14f778d23adaea4bebe43f7ebcfe
Successfully built autocorrect
processing:  project_essay_1
processing:  project_essay_2
processing:  project_resource_summary
processing:  resource_description
processing:  project_title
CPU times: user 26min 15s, sys: 6.06 s, total: 26min 21s
Wall time: 26min 27s


In [0]:
#Categorical Features
def getCatFeatures(T, Col):
    vectorizer = CountVectorizer(binary=True,
                                 ngram_range=(1,1),
                                 tokenizer=lambda x:[a.strip() for a in x.split(',')])
    return vectorizer.fit_transform(T[Col].fillna(''))

X_tp = getCatFeatures(df, 'teacher_prefix')
X_ss = getCatFeatures(df, 'school_state')
X_pgc = getCatFeatures(df, 'project_grade_category')
X_psc = getCatFeatures(df, 'project_subject_categories')
X_pssc = getCatFeatures(df, 'project_subject_subcategories')
df['Month']=str(df['Month'])
df['Weekday']=str(df['Weekday'])
X_month = getCatFeatures(df, 'Month')
X_weekday = getCatFeatures(df, 'Weekday')

X_cat = hstack((X_tp, X_ss, X_pgc, X_psc, X_pssc,X_month,X_weekday))

del X_tp, X_ss, X_pgc, X_psc, X_pssc, X_month, X_weekday

In [26]:
df.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'Ind', 'items', 'quantity', 'price', 'total_price', 'min_quantity',
       'min_price', 'min_total_price', 'max_quantity', 'max_price',
       'max_total_price', 'mean_quantity', 'mean_price', 'mean_total_price',
       'resource_description', 'Year', 'Month', 'Weekday', 'Month_Day',
       'Weekend_Flag', 'approvedValue', 'four_essays', 'text', 'char_count',
       'word_count', 'word_density', 'punctuation_count', 'title_word_count',
       'upper_case_word_count', 'stopword_count', 'title_len', 'e1_len',
       'e2_len', 'ressum_len', 'title_polarity', 'title_subjectivity',
  

In [27]:
numFeatures=['teacher_number_of_previously_posted_projects','items','quantity','price','total_price','min_quantity','min_price','min_total_price','max_quantity','max_price','max_total_price',
             'mean_quantity','mean_price','mean_total_price','Year','Month_Day','Weekend_Flag','approvedValue','four_essays','char_count',
             'word_count','word_density','punctuation_count','title_word_count','upper_case_word_count','stopword_count',
             'e1_len','e2_len','title_len','ressum_len','title_polarity', 'title_subjectivity','e1_polarity', 'e1_subjectivity', 'e2_polarity',
             'e2_subjectivity','ressum_polarity','ressum_subjectivity']

df[numFeatures].head()

,teacher_number_of_previously_posted_projects,items,quantity,price,total_price,min_quantity,min_price,min_total_price,max_quantity,max_price,max_total_price,mean_quantity,mean_price,mean_total_price,Year,Month_Day,Weekend_Flag,approvedValue,four_essays,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,stopword_count,e1_len,e2_len,title_len,ressum_len,title_polarity,title_subjectivity,e1_polarity,e1_subjectivity,e2_polarity,e2_subjectivity,ressum_polarity,ressum_subjectivity
0,26,2,6,299.98,899.94,3,149.99,449.97,3,149.99,449.97,3.0,149.990,449.970,2016,18,0,11102621.33,0,2080,365,5.683060,52,39,9,158,170,142,4,21,0.333333,0.666667,0.249909,0.439385,0.144444,0.300000,0.400000,0.700000
1,1,1,20,20.00,400.00,20,20.00,400.00,20,20.00,400.00,20.0,20.000,400.000,2017,26,0,0.00,0,1407,220,6.366516,43,31,4,85,90,100,5,12,0.300000,0.750000,0.263750,0.717500,0.111905,0.459524,0.000000,0.000000
2,5,1,1,469.99,469.99,1,469.99,469.99,1,469.99,469.99,1.0,469.990,469.990,2017,1,1,26350156.08,0,1552,276,5.602888,29,36,5,119,133,101,4,33,0.000000,0.000000,0.345739,0.487879,0.367857,0.614286,1.000000,0.300000
3,16,5,5,684.47,684.47,1,18.95,18.95,1,354.99,354.99,1.0,136.894,136.894,2016,12,0,12599720.68,0,2871,459,6.241304,94,43,26,208,197,189,10,36,0.500000,0.500000,0.162302,0.391270,0.193033,0.447744,-0.041667,0.372619
4,42,1,2,355.50,711.00,2,355.50,711.00,2,355.50,711.00,2.0,355.500,711.000,2016,6,1,25667613.43,0,1199,218,5.474886,17,23,1,107,80,105,9,12,0.183333,0.350000,0.407143,0.606122,0.190741,0.519136,0.000000,0.000000


In [0]:
from sklearn.preprocessing import StandardScaler
X = hstack((X_txt, X_cat, StandardScaler().fit_transform(df[numFeatures].fillna(0)))).tocsr()

In [0]:

#Saving the combined features into an NPZ file
from scipy import sparse
sparse.save_npz("pred_X.npz", X)

In [0]:
X = sparse.load_npz("pred_X.npz")


In [0]:
x_train = X[df.index[df.Ind == 'Train'], :]
x_test = X[df.index[df.Ind == 'Test'], :]

y_train = df[df.Ind=='Train']['project_is_approved'].values
y_test = df[df.Ind=='Test'][['id','project_is_approved']]

In [0]:

param_test ={'num_leaves': sp_randint(6, 50),
             'min_child_samples': sp_randint(100, 500),
             'min_child_weight': [1e-5, 1e-3,  1e-1, 1, 1e1, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8),
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 10, 20, 50, 100]}

n_HP_points_to_test = 100

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=123, silent=True, metric='None', n_jobs=-1, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test,
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=123,n_jobs=-1,verbose=1)

In [0]:

gs.fit(x_train, y_train)
gs.best_estimator_


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [0]:

lgbmModelBest = gs.best_estimator_
lgbmModelBest.fit(x_train, y_train)

In [0]:
pred = lgbmModelBest.predict_proba(x_test)[:,1]
results=x_test[['id']]
results['project_is_approved']=pred
results.to_csv("/content/submission_v3_lgbm.csv",index=False)